# ETL

Het extract - transform - load concept is een veel voorkomend begrip in (big) data toepassingen en geeft het stappenplan weer van de levenscyclus van de data binnen je toepassing.
Het concept bestaat uit drie stappen:
* extract: zoeken van data, inlezen en validatie
* transform: verwerken van data, data cleaning, aggregatie, groupering, filtering, ...
* load: opslaan van de getransformeerde data in een file, database, datawarehouse, datalake, ...

In de rest van deze notebook gaan we bestuderen hoe deze stappen uit te voeren met Spark.
Hiervoor gaan we een csv gebruiken als bronbestand.

## Extract

In deze directory staat een zip file waarin deze csv is opgeslaan. 
Unzip deze file eerst en upload het naar het hdfs

In [2]:
import zipfile
from hdfs import InsecureClient

with zipfile.ZipFile("cars.zip", 'r') as zip_ref:
    zip_ref.extractall()

client = InsecureClient('http://localhost:9870', user='bigdata')
map = "/user/bigdata/ETL"
filename = 'cars.csv'

if client.status(map, strict=False) is None:
    client.makedirs(map)
else:
    # do some cleaning in case anything else than *.txt is present
    for f in client.list(map):
        client.delete(map + '/' + f, recursive=True)

client.upload(map, filename)

'/user/bigdata/ETL/cars.csv'

Maak nu een locale sparkcontext aan en lees dit bestand in

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ETL").getOrCreate()

df = spark.read.csv(map + "/" + filename, header=True, sep=",").cache()
print('Total Records = {}'.format(df.count()))
df.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/07 12:47:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/07 12:47:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


De datastructuur van het csv is als volgt:

In [3]:
df.printSchema()

root
 |-- manufacturer_name: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- color: string (nullable = true)
 |-- odometer_value: string (nullable = true)
 |-- year_produced: string (nullable = true)
 |-- engine_fuel: string (nullable = true)
 |-- engine_has_gas: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- engine_capacity: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- has_warranty: string (nullable = true)
 |-- state: string (nullable = true)
 |-- drivetrain: string (nullable = true)
 |-- price_usd: string (nullable = true)
 |-- is_exchangeable: string (nullable = true)
 |-- location_region: string (nullable = true)
 |-- number_of_photos: string (nullable = true)
 |-- up_counter: string (nullable = true)
 |-- feature_0: string (nullable = true)
 |-- feature_1: string (nullable = true)
 |-- feature_2: string (nullable = true)
 |-- feature_3: string (nullable = true)


## Transform

De transform stap is de meest complexe stap van de drie en kan uit een grote verscheidenheid van bewerkingen bestaan, zoals:
* Dataformaten aanpassen
* Vertalingen van tekst
* Geencodeerde waarden aanpassen: 0/1 vs true/false of m/f vs male/female
* Allerhande data-cleaning stappen
* Encoderen (Ordinal of One-hot) van categorieke kolommen
* Groeperen van data
* Uitvoeren van berekeningen 
* ...

Schrijf hieronder eerst zelf de code om de volgende stappen uit te voeren:
* Omzetten naar integer van de kolommen: odometer_value, year_produced, engine_capacity, price_usd, number_of_photos, up_counter, duration_listed
* Omzetten naar boolean van de kolommen: engine_has_gas, has_warranty, is_exchangeable, feature_0 tot en met 9
* Bereken het aantal null en nan waarden per kolom

In [4]:
df_backup = df

In [5]:
from pyspark.sql.types import IntegerType, BooleanType
from pyspark.sql.functions import col, isnan, isnull, when, count

In [6]:
df = df_backup

# cast to integers
df = df.withColumn("odometer_value",col("odometer_value").cast(IntegerType()))
df = df.withColumn("year_produced",col("year_produced").cast(IntegerType()))
df = df.withColumn("engine_capacity",col("engine_capacity").cast(IntegerType()))
df = df.withColumn("price_usd",col("price_usd").cast(IntegerType()))
df = df.withColumn("number_of_photos",col("number_of_photos").cast(IntegerType()))
df = df.withColumn("up_counter",col("up_counter").cast(IntegerType()))
df = df.withColumn("duration_listed",col("duration_listed").cast(IntegerType()))

# cast to booleans
cols = ["engine_has_gas","has_warranty","is_exchangeable","feature_0","feature_1","feature_2","feature_3","feature_4","feature_5","feature_6","feature_7","feature_8","feature_9"]
for c in cols:
    df = df.withColumn(c, col(c).cast(BooleanType()))
    
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------------+----------+------------+-----+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------------+
|manufacturer_name|model_name|transmission|color|odometer_value|year_produced|engine_fuel|engine_has_gas|engine_type|engine_capacity|body_type|has_warranty|state|drivetrain|price_usd|is_exchangeable|location_region|number_of_photos|up_counter|feature_0|feature_1|feature_2|feature_3|feature_4|feature_5|feature_6|feature_7|feature_8|feature_9|duration_listed|
+-----------------+----------+------------+-----+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+-------

In [7]:
df.printSchema()

root
 |-- manufacturer_name: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- color: string (nullable = true)
 |-- odometer_value: integer (nullable = true)
 |-- year_produced: integer (nullable = true)
 |-- engine_fuel: string (nullable = true)
 |-- engine_has_gas: boolean (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- engine_capacity: integer (nullable = true)
 |-- body_type: string (nullable = true)
 |-- has_warranty: boolean (nullable = true)
 |-- state: string (nullable = true)
 |-- drivetrain: string (nullable = true)
 |-- price_usd: integer (nullable = true)
 |-- is_exchangeable: boolean (nullable = true)
 |-- location_region: string (nullable = true)
 |-- number_of_photos: integer (nullable = true)
 |-- up_counter: integer (nullable = true)
 |-- feature_0: boolean (nullable = true)
 |-- feature_1: boolean (nullable = true)
 |-- feature_2: boolean (nullable = true)
 |-- feature_3: boolean (null

In bovenstaande code kan je zien dat er slechts een aantal null-waarden in de dataset aanwezig zijn.
Deze kunnen ingevuld worden door middel van een [imputer](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.Imputer.html).
Hier laten we deze rijen echter gewoon vallen voor de eenvoud:

In [8]:
df = df.na.drop()
df.show()

+-----------------+----------+------------+------+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------------+
|manufacturer_name|model_name|transmission| color|odometer_value|year_produced|engine_fuel|engine_has_gas|engine_type|engine_capacity|body_type|has_warranty|state|drivetrain|price_usd|is_exchangeable|location_region|number_of_photos|up_counter|feature_0|feature_1|feature_2|feature_3|feature_4|feature_5|feature_6|feature_7|feature_8|feature_9|duration_listed|
+-----------------+----------+------------+------+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+----

De oefening om de waarden in te vullen met een imputer (bvb door het gemiddelde) kan je hieronder doen.

In [9]:
# oefening

Bereken nu de volgende waarden van de beschikbare data:
* Aantal autos per merk
* Welke verschillende types van transmissie zijn er?
* Marktaandeel (percentage) van de verschillende types motor?
* Maximum prijs van elk merk
* Wat zijn de vijf goedkoopste voertuigen met een automatische transmissie?

In [22]:
# autos per merk
df_autos = df.groupBy("manufacturer_name").count()
df_autos.show()

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|         Infiniti|  162|
|          Peugeot| 1909|
|            Lexus|  213|
|           Jaguar|   53|
|            Rover|  235|
|           Lancia|   92|
|             Jeep|  107|
|       Mitsubishi|  887|
|              ГАЗ|  200|
|              Kia|  912|
|             Mini|   68|
|        Chevrolet|  435|
|            Volvo|  721|
|            Lifan|   47|
|          Hyundai| 1116|
|             Saab|  108|
|             LADA|  146|
|            Honda|  797|
|        SsangYong|   79|
+-----------------+-----+
only showing top 20 rows



In [11]:
# types transmissie
df.select("transmission").distinct().show()

+------------+
|transmission|
+------------+
|   automatic|
|  mechanical|
+------------+



In [12]:
# marktaandeel
totaal = df.count()
df_aandeel = df.groupBy("engine_type").count()
df_aandeel = df_aandeel.withColumn("marktaandeel", col("count")/totaal)
df_aandeel.show()

+-----------+-----+-------------------+
|engine_type|count|       marktaandeel|
+-----------+-----+-------------------+
|   gasoline|25647| 0.6657926845097479|
|     diesel|12874|0.33420731549025207|
+-----------+-----+-------------------+



In [13]:
# maximum prijs per merk
df.groupBy("manufacturer_name").max("price_usd").show()

+-----------------+--------------+
|manufacturer_name|max(price_usd)|
+-----------------+--------------+
|       Volkswagen|         43999|
|         Infiniti|         47000|
|          Peugeot|         20450|
|            Lexus|         48610|
|           Jaguar|         50000|
|            Rover|          9900|
|           Lancia|          9500|
|             Jeep|         43000|
|       Mitsubishi|         31400|
|              ГАЗ|         30000|
|              Kia|         44700|
|             Mini|         39456|
|        Chevrolet|         49900|
|            Volvo|         48200|
|            Lifan|         15750|
|          Hyundai|         45954|
|             Saab|         17500|
|             LADA|         13800|
|            Honda|         42000|
|        SsangYong|         15900|
+-----------------+--------------+
only showing top 20 rows



In [20]:
# goedkoopste voertuigen met automatische transmissie
df_result = df.filter(col("transmission") == "automatic").sort(col("price_usd").asc()).limit(5)

## Load

In deze stap veronderstellen we dat we enkel de 5 goedkoopste auto's willen bewaren.
Schrijf hieronder de benodigde code om de informatie van deze autos op te slaan in een json.

In [21]:
df_result.write.format('json').save('ETL/result.json')

Dit is een voorbeeld waarbij de resultaten worden opgeslaan in een bestand.
Andere mogelijkheden zijn om het op te slaan in een SQL-database.
Demo-code om dit te bereiken kan je [hier](https://kontext.tech/column/spark/395/save-dataframe-to-sql-databases-via-jdbc-in-pyspark) bekijken.
Later in dit vak zullen we ook NoSQL-databases bekijken.
Op dat moment zullen we zien hoe we de resultaten kunnen bewaren in dit type database beheersystemen (DBMS).